均方差：Mean Squared Error(简写为：MSE)

$ \sum \left (  y - \bar{y}\right )^2$

其中，$\bar{y}$ 是模型的预测值

线性均方差：$loss = \sum \left [ y - \left ( xw + b \right )  \right ] ^2$

它长得有些像L2范数但是它没有开根号

L2-norm = $\left \| y - \left( xw+b \right) \right \| _{_{2} } $

所以，线性均方差也可以等于它的L2范数的平方

$loss = norm \left( y - \left( xw + b \right ) \right)^2$

torch.norm(y - $Pred$, 2).pow(2)

# MSE的求导

![](./img/mse.png)

## pytorch的自动求导(autograd.grad)

In [6]:
import torch
from torch.nn import functional as F

# Pred = x w + b
x = torch.ones(1)  # 其中x初始化为1，w
w = torch.full([1], 2.)  # w初始化为dim = 1，长度为1的tensor
# 没有使用b，这里b = 0
mse = F.mse_loss(torch.ones(1), x * w)  # 第一个参数为Pred, 第二个为label，因为是平方所以反过来写也没问题，但建议按顺序写
# （1 - 2）^2 = 1
mse

tensor(1.)

In [7]:
torch.autograd.grad(mse, [w])  # 第一个参数是Pred(y), 第二个参数是[w1, w2, w3, ……]（[x1, x2, x3, ……]）

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

报错意思是你的w是不需要求导的但你求了导

出现这个错误的根本原因是初始化w的时候没有设置为需要求导信息

In [8]:
w.requires_grad_() # 对w进行更新，说明w需要求导

tensor([2.], requires_grad=True)

### requires_grad_与requires_grad的区别（摘自ChatGPT）

在PyTorch中，`requires_grad`是一个布尔值，它指示一个张量是否需要计算梯度。当`requires_grad=True`时，PyTorch会跟踪该张量的计算历史并计算梯度，以便进行反向传播以进行自动微分。默认情况下，`requires_grad`为`False`。

`requires_grad_`是一个in-place方法，它可以用来更改一个张量的`requires_grad`属性。与`requires_grad`不同，`requires_grad_`允许您直接修改张量的属性，而不是创建一个新的张量。例如，如果您有一个张量`x`，您可以使用`x.requires_grad_(True)`来将`x`的`requires_grad`属性设置为`True`。

需要注意的是，`requires_grad_`方法返回的是原始张量本身，并且如果对原始张量进行in-place操作，则梯度也会相应地更改。因此，在使用`requires_grad_`方法时需要小心，以确保梯度计算的正确性。

In [10]:
torch.autograd.grad(mse, [w])
# 出错是因为mse中的w没更新

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [9]:
mse = F.mse_loss(torch.ones(1), x * w)
torch.autograd.grad(mse, [w])

(tensor([2.]),)